In [ ]:
pip install pyDOE

In [ ]:
import time
import torch
import numpy as np
import scipy.io
from pyDOE import lhs
from physicsinformed_1D import PhysicsInformedContinuous
from scipy.interpolate import griddata
import utilities_1D

# Select gpu if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dtype = torch.float

# Set seed for the Random Number Generator (RNG), setting the seed generates same set of random numbers everytime 
torch.manual_seed(0)
np.random.seed(0) 

# Define no. of training points
N0 = 100
N_b = 50
N_f = 20000

# Define feed-forward network architecture
layers = [2, 40, 40, 40, 40, 40, 1]

# Define no. of epochs for each optimizer
epochs_Adam = 200
epochs_LBFGS = 100

### PRE-PROCESSING ###
# Loading benchmark data
data = scipy.io.loadmat('1D_datapoints.mat')
t = data['t'].flatten()[:, None]
x = data['x'].flatten()[:, None]
u_sol = data['u'].T

X, T = np.meshgrid(x, t) # Creates mesh such that for every xi, all the ti s can be accessed and vice-versa

# Transform grid into vectors that can be processed by the neural net
X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None])) #Stack arrays in sequence horizontally (column wise)
u_star = u_sol.flatten()[:, None]

# Domain bounds
lb = X_star.min(0)
ub = X_star.max(0)

# Select random data points for the initial condition
idx_x = np.random.choice(x.shape[0], N0, replace=False) # Choose random 100 indices from 256 total indices of space
x0 = x[idx_x, :] # location of these 100 random points
u0 = torch.tensor(u_sol.T[idx_x, 0:1], dtype=dtype, device=device) # Choose temperatures of 100 random points at t=0

# Select random data points for the boundary condition
idx_t = np.random.choice(t.shape[0], N_b, replace=False) # Choose random 50 indices from 201 total indices of time
tb = t[idx_t, :] # time stamps of those 50 indices

# Create collocation points with latin hypercube sampling
X_f = lb + (ub - lb) * lhs(2, N_f) # Creates 20000 sample points in both x and t

X0 = np.concatenate((x0, 0 * x0), 1) # (x0, 0)
X_lb = np.concatenate((0 * tb + lb[0], tb), 1) # (lb[0], tb)
X_ub = np.concatenate((0 * tb + ub[0], tb), 1) # (ub[0], tb)

### TRAINING ###
# Create torch.tensors of training data
x0 = torch.tensor(X0[:, 0:1], dtype=dtype, requires_grad=True, device=device)
t0 = torch.tensor(X0[:, 1:2], dtype=dtype, requires_grad=True, device=device)

x_lb = torch.tensor(X_lb[:, 0:1], dtype=dtype, requires_grad=True, device=device)
t_lb = torch.tensor(X_lb[:, 1:2], dtype=dtype, requires_grad=True, device=device)

x_ub = torch.tensor(X_ub[:, 0:1], dtype=dtype, requires_grad=True, device=device)
t_ub = torch.tensor(X_ub[:, 1:2], dtype=dtype, requires_grad=True, device=device)

x_f = torch.tensor(X_f[:, 0:1], dtype=dtype, requires_grad=True, device=device)
t_f = torch.tensor(X_f[:, 1:2], dtype=dtype, requires_grad=True, device=device)

# Initialize PINN model
PINNModel = PhysicsInformedContinuous(layers, t0, x0, t_lb, x_lb, t_ub, x_ub, t_f, x_f, u0)

# Train the model
start_time = time.time()
PINNModel.train(epochs_Adam, optimizer='Adam', lr=0.001)
#PINNModel.train(epochs_LBFGS, optimizer='L-BFGS')
elapsed = time.time() - start_time
print('Training time: %.4f' % (elapsed))

# Create torch.tensors to predict solution for the whole domain
x_star = torch.tensor(X_star[:, 0:1], dtype=dtype, requires_grad=True, device=device)
t_star = torch.tensor(X_star[:, 1:2], dtype=dtype, requires_grad=False, device=device)

# Predict temperature distribution and first derivative for the heat flux
u_pred = PINNModel.u_nn(t_star, x_star)
u_x_pred = utilities_1D.get_derivative(u_pred, x_star, 1)

### POST-PROCESSING ###
u_pred = u_pred.detach().cpu().numpy()
u_x_pred = u_x_pred.detach().cpu().numpy()


# Compute error measure
error_u = np.linalg.norm(u_star - u_pred, 2) / np.linalg.norm(u_star, 2)
print('Error u: %e' % (error_u))

u_pred_grid = griddata(X_star, u_pred.flatten(), (X, T), method='cubic')
error_u_abs = np.abs(u_sol - u_pred_grid)

X_u_train = np.vstack([X0, X_lb, X_ub])
utilities_1D.plot_results(t, x, u_pred_grid, u_sol, X_u_train, lb, ub)

# Plot training history and predicitons
PINNModel.plot_training_history()

Sequential(
  (0): Linear(in_features=2, out_features=40, bias=True)
  (1): Tanh()
  (2): Linear(in_features=40, out_features=40, bias=True)
  (3): Tanh()
  (4): Linear(in_features=40, out_features=40, bias=True)
  (5): Tanh()
  (6): Linear(in_features=40, out_features=40, bias=True)
  (7): Tanh()
  (8): Linear(in_features=40, out_features=40, bias=True)
  (9): Tanh()
  (10): Linear(in_features=40, out_features=1, bias=True)
)
model parameters on gpu: False
Epoch (Adam): 0, Cost: 0.01746806502342224
Epoch (Adam): 100, Cost: 0.00018169693066738546
